In [1]:
model_config = {
    'task': 'Congestion',
    'save_path': './UNet_Inception/',
    'pretrained': None,
    'max_iters': 200000,
    'plot_roc': False,
    'arg_file': None,
    'cpu': False,
    'dataroot': '/home/palaniappan-r/Repos/boosted-UNet/training_datasets/congestion_train_set/congestion',
    'ann_file_train': './files/train.csv',
    'ann_file_test': './files/test.csv',
    'dataset_type': 'CongestionDataset',
    'batch_size': 4,
    'aug_pipeline': ['Flip'],
    'model_type': 'Congestion_Prediction_Net',
    'in_channels': 3,
    'out_channels': 1,
    'lr': 0.0002,
    'weight_decay': 0.0001,
    'loss_type': 'MSELoss',
    'eval_metric': ['NRMS', 'SSIM'],
    'ann_file': './files/train.csv',
    'test_mode': False
}


### Dataset Building

Defines a set of classes and functions to create and manage data augmentation and loading pipelines for training machine learning models. The key components are:

1. **Flip Class**:
    - Provides a data augmentation method that flips images either horizontally or vertically with a given probability.
    - The `__call__` method checks the flip ratio and performs the flip if the condition is met.

2. **Rotation Class**:
    - Provides a data augmentation method that rotates images by 90 degrees in any of the four possible directions with a given probability.
    - The `__call__` method checks the rotate ratio and performs the rotation if the condition is met.

3. **IterLoader Class**:
    - Wraps around a `DataLoader` to provide an iterable loader that handles the `StopIteration` exception by resetting the iterator.
    - Implements `__next__`, `__len__`, and `__iter__` methods for iteration control.

4. **build_dataset Function**:
    - Constructs a dataset with specified augmentation methods based on the `train_options` configuration.
    - If augmentation methods are specified and not in test mode, constructs a pipeline of these methods.
    - Retrieves the dataset class from the `datasets` module and initializes it with the provided options and augmentation pipeline.
    - Returns a `DataLoader` or `IterLoader` depending on whether it is in test mode or not.

In [2]:
import time
import numpy as np
import mmcv
from torch.utils.data import DataLoader
import datasets

class Flip:
    _directions = ['horizontal', 'vertical']

    def __init__(self, keys=['feature', 'label'], flip_ratio=0.5, direction='horizontal', **kwargs):
        if direction not in self._directions:
            raise ValueError(f'Direction {direction} is not supported. Currently supported directions are {self._directions}')
        self.keys = keys
        self.flip_ratio = flip_ratio
        self.direction = direction

    def __call__(self, results):
        if np.random.random() < self.flip_ratio:
            for key in self.keys:
                if isinstance(results[key], list):
                    for v in results[key]:
                        mmcv.imflip_(v, self.direction)
                else:
                    mmcv.imflip_(results[key], self.direction)
        return results

class Rotation:
    def __init__(self, keys=['feature', 'label'], axis=(0, 1), rotate_ratio=0.5, **kwargs):
        self.keys = keys
        self.axis = {k: axis for k in keys} if isinstance(axis, tuple) else axis
        self.rotate_ratio = rotate_ratio
        self.directions = [0, -1, -2, -3]

    def __call__(self, results):
        if np.random.random() < self.rotate_ratio:
            rotate_angle = self.directions[int(np.random.random() * 4)]
            for key in self.keys:
                if isinstance(results[key], list):
                    for v in results[key]:
                        v = np.ascontiguousarray(np.rot90(v, rotate_angle, axes=self.axis[key]))
                else:
                    results[key] = np.ascontiguousarray(np.rot90(results[key], rotate_angle, axes=self.axis[key]))
        return results


class IterLoader:
    def __init__(self, dataloader):
        self._dataloader = dataloader
        self.iter_loader = iter(self._dataloader)

    def __next__(self):
        try:
            data = next(self.iter_loader)
        except StopIteration:
            time.sleep(1)
            self.iter_loader = iter(self._dataloader)
            data = next(self.iter_loader)
        return data

    def __len__(self):
        return len(self._dataloader)

    def __iter__(self):
        return self


def build_dataset(train_options):
    aug_methods = {
        'Flip': Flip(),
        'Rotation': Rotation(**train_options)
    }

    if 'aug_pipeline' in train_options and not train_options['test_mode']:
        pipeline = [aug_methods[method] for method in train_options.pop('aug_pipeline')]
    else:
        pipeline = None

    dataset_cls = datasets.__dict__[train_options.pop('dataset_type')]
    dataset = dataset_cls(**train_options, pipeline=pipeline)

    if train_options['test_mode']:
        return DataLoader(dataset=dataset, num_workers=1, batch_size=1, shuffle=False)
    else:
        return IterLoader(DataLoader(
            dataset=dataset,
            num_workers=1,
            batch_size=train_options.pop('batch_size'),
            shuffle=True,
            drop_last=True,
            pin_memory=True
        ))


In [3]:
dataset = build_dataset(model_config)

### Inception Encoder

Classes for the inception encoder

1. **generation_init_weights Function**:
    - Initializes the weights of neural network layers.
    - Applies normal distribution initialization to weights and zero initialization to biases for layers of type `Conv` and `Linear`.

2. **CreateEncoderSingleConv Class**:
    - Defines a single convolutional layer block.
    - Comprises a convolutional layer, batch normalization, and PReLU activation.
    - Ensures the kernel size is odd for padding purposes.

3. **EncoderInceptionModuleSingle Class**:
    - Defines an inception-like module with multiple convolutional paths.
    - Includes bottleneck convolution followed by multiple convolutional layers with different kernel sizes (1, 3, 5, and 7) and max pooling layers.

4. **EncoderModule Class**:
    - Constructs a sequential module comprising multiple encoder blocks.
    - Can use either inception modules or simple convolutional blocks based on the `use_inception` flag.
    - Repeats the specified number of times (`repeat_num`).

5. **IncepEncoder Class**:
    - Defines an encoder network using the `EncoderModule`.
    - Configures the encoder with a specified number of repeated modules and optional inception usage.
    - Sets a global variable for prediction type and includes a method for weight initialization using `generation_init_weights`.

In [4]:
import torch.nn as nn

def generation_init_weights(module):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            if m.weight is not None:
                nn.init.normal_(m.weight, 0.0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    module.apply(init_func)

class CreateEncoderSingleConv(nn.Module):
    def __init__(self, in_chs, out_chs, kernel):
        super().__init__()
        assert kernel % 2 == 1, "Kernel size must be odd"

        self.single_conv = nn.Sequential(
            nn.Conv2d(in_chs, out_chs, kernel_size=kernel, padding=(kernel - 1) // 2),
            nn.BatchNorm2d(out_chs),
            nn.PReLU(num_parameters=out_chs)
        )

    def forward(self, x):
        return self.single_conv(x)


class EncoderInceptionModuleSingle(nn.Module):
    def __init__(self, channels):
        super().__init__()
        assert channels % 2 == 0, "Channels must be even"

        bn_ch = channels // 2
        self.bottleneck = CreateEncoderSingleConv(channels, bn_ch, 1)
        self.conv1 = CreateEncoderSingleConv(bn_ch, channels, 1)
        self.conv3 = CreateEncoderSingleConv(bn_ch, channels, 3)
        self.conv5 = CreateEncoderSingleConv(bn_ch, channels, 5)
        self.conv7 = CreateEncoderSingleConv(bn_ch, channels, 7)

        self.pool3 = nn.MaxPool2d(3, stride=1, padding=1)
        self.pool5 = nn.MaxPool2d(5, stride=1, padding=2)

    def forward(self, x):
        bn = self.bottleneck(x)
        out = self.conv1(bn) + self.conv3(bn) + self.conv5(bn) + self.conv7(bn) + self.pool3(x) + self.pool5(x)
        return out


class EncoderModule(nn.Module):
    def __init__(self, chs, repeat_num, use_inception):
        super().__init__()
        if use_inception:
            layers = [EncoderInceptionModuleSingle(chs) for _ in range(repeat_num)]
        else:
            layers = [CreateEncoderSingleConv(chs, chs, 3) for _ in range(repeat_num)]
        self.convs = nn.Sequential(*layers)

    def forward(self, x):
        return self.convs(x)


class IncepEncoder(nn.Module):
    def __init__(self, use_inception, repeat_per_module, prediction_type, middle_layer_size=256):
        super().__init__()
        self.encoderPart = EncoderModule(middle_layer_size, repeat_per_module, use_inception)
        global g_prediction_task
        g_prediction_task = prediction_type

    def forward(self, x):
        return self.encoderPart(x)

    def init_weights(self):
        """Initialize the weights."""
        generation_init_weights(self)



### U-Net

The U-Net model has an encoder-decoder architecture. The encoder gradually downsamples the input image, while the decoder upsamples it back to the original resolution. Skip connections are used to concatenate the feature maps from the encoder to the corresponding feature maps in the decoder, helping to retain spatial information.

Defines a U-Net architecture with additional inception-like modules for enhanced feature extraction. The components include:

1. **DoubleConv Class**:
    - Implements two consecutive convolutional layers, each followed by batch normalization and PReLU activation.
    - Enhances feature extraction by applying two sets of convolutional operations.

2. **Down Class**:
    - Defines a downsampling module consisting of a max-pooling layer followed by a `DoubleConv` layer.
    - Reduces the spatial dimensions while increasing the feature channels.

3. **Up Class**:
    - Defines an upsampling module using bilinear upsampling followed by a `DoubleConv` layer.
    - Concatenates the upsampled feature map with a corresponding feature map from the contracting path to preserve spatial information.

4. **OutConv Class**:
    - Implements a single convolutional layer to produce the final output with the desired number of channels.

5. **UNet Class**:
    - Constructs the U-Net architecture with downsampling and upsampling paths.
    - Incorporates an inception-like encoder module (`IncepEncoder`) at the bottleneck to enhance feature representation.
    - Defines the forward pass, which includes downsampling, inception module processing, and upsampling to generate the final output.
    - Provides a method to initialize the weights of the network layers using `generation_init_weights`.

6. **init_weights Method**:
    - Initializes the weights of the U-Net layers using normal distribution for weights and zero initialization for biases.

In [5]:
import torch
import torch.nn as nn


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.PReLU(num_parameters=out_channels),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.PReLU(num_parameters=out_channels)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True, task="UnKnown"):
        global g_prediction_task
        super().__init__()
        g_prediction_task = task
        _SCALE_ = 2

        self.inc = DoubleConv(n_channels, 64 // _SCALE_)
        self.down1 = Down(64 // _SCALE_, 128 // _SCALE_)
        self.down2 = Down(128 // _SCALE_, 256 // _SCALE_)
        self.down3 = Down(256 // _SCALE_, 512 // _SCALE_)
        self.down4 = Down(512 // _SCALE_, 512 // _SCALE_)
        self.up1 = Up(1024 // _SCALE_, 256 // _SCALE_, bilinear)
        self.up2 = Up(512 // _SCALE_, 128 // _SCALE_, bilinear)
        self.up3 = Up(256 // _SCALE_, 64 // _SCALE_, bilinear)
        self.up4 = Up(128 // _SCALE_, 64 // _SCALE_, bilinear)
        self.outc = OutConv(64 // _SCALE_, n_classes)

        self.incepEncoder = IncepEncoder(True, 1, g_prediction_task, 512 // _SCALE_)
        self.Conv4Inception = DoubleConv(512 // _SCALE_, 512 // _SCALE_)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x5B = self.incepEncoder(x5)  # Insert the Inception Module at the bottleneck
        x5C = self.Conv4Inception(x5B)
        x = self.up1(x5C, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    def init_weights(self):
        """Initialize the weights."""
        generation_init_weights(self)



In [6]:
import torch
import torch.nn as nn
from collections import OrderedDict

def generation_init_weights(module):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and ('Conv' in classname or 'Linear' in classname):
            if m.weight is not None:
                nn.init.normal_(m.weight, 0.0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
    module.apply(init_func)

def load_state_dict(module, state_dict, strict=False, logger=None):
    unexpected_keys = []
    all_missing_keys = []
    err_msg = []

    metadata = getattr(state_dict, '_metadata', None)
    state_dict = state_dict.copy()
    if metadata is not None:
        state_dict._metadata = metadata

    def load(module, prefix=''):
        local_metadata = {} if metadata is None else metadata.get(prefix[:-1], {})
        module._load_from_state_dict(state_dict, prefix, local_metadata, True,
                                     all_missing_keys, unexpected_keys, err_msg)
        for name, child in module._modules.items():
            if child is not None:
                load(child, prefix + name + '.')

    load(module)
    load = None

    missing_keys = [key for key in all_missing_keys if 'num_batches_tracked' not in key]

    if unexpected_keys:
        err_msg.append(f'unexpected key in source state_dict: {", ".join(unexpected_keys)}\n')
    if missing_keys:
        err_msg.append(f'missing keys in source state_dict: {", ".join(missing_keys)}\n')

    if err_msg:
        err_msg.insert(0, 'The model and loaded state dict do not match exactly\n')
        err_msg = '\n'.join(err_msg)
        if strict:
            raise RuntimeError(err_msg)
        elif logger is not None:
            logger.warning(err_msg)
        else:
            print(err_msg)
    return missing_keys

class PredictionNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, **kwargs):
        super().__init__()

    def forward(self, x):
        pass

    def init_weights(self, pretrained=None, strict=True, **kwargs):
        if isinstance(pretrained, str):
            new_dict = OrderedDict()
            weight = torch.load(pretrained, map_location='cpu')['state_dict']
            for k, v in weight.items():
                new_dict[k] = v
            load_state_dict(self, new_dict, strict=strict, logger=None)
        elif pretrained is None:
            generation_init_weights(self)
        else:
            raise TypeError("'pretrained' must be a str or None.")

class Congestion_Prediction_Net(PredictionNet):
    def __init__(self, in_channels=3, out_channels=1, **kwargs):
        super().__init__()
        self.uNet = UNet(n_channels=3, n_classes=1, bilinear=True, task="Congestion")

    def forward(self, x):
        return self.uNet(x)

    def init_weights(self, pretrained=None, strict=True, **kwargs):
        super().init_weights(pretrained, strict)


In [7]:
model = Congestion_Prediction_Net()
model.init_weights()
model = model.cuda()
print(model)

Congestion_Prediction_Net(
  (uNet): UNet(
    (inc): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=32)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): PReLU(num_parameters=32)
      )
    )
    (down1): Down(
      (maxpool_conv): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): DoubleConv(
          (double_conv): Sequential(
            (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): PReLU(num_parameters=64)
            (3): Conv2d(64, 64, kernel_size

In [8]:
# Copyright 2022 CircuitNet. All rights reserved.

import functools

import torch.nn as nn
import torch.nn.functional as F

__all__ = ['L1Loss', 'MSELoss']


def build_loss(opt):
    return globals()[opt.pop('loss_type')]()

def reduce_loss(loss, reduction):
    reduction_enum = F._Reduction.get_enum(reduction)
    if reduction_enum == 0:
        return loss
    if reduction_enum == 1:
        return loss.mean()

    return loss.sum()


def mask_reduce_loss(loss, weight=None, reduction='mean', sample_wise=False):
    if weight is not None:
        assert weight.dim() == loss.dim()
        assert weight.size(1) == 1 or weight.size(1) == loss.size(1)
        loss = loss * weight

    if weight is None or reduction == 'sum':
        loss = reduce_loss(loss, reduction)
    elif reduction == 'mean':
        if weight.size(1) == 1:
            weight = weight.expand_as(loss)
        eps = 1e-12

        if sample_wise:
            weight = weight.sum(dim=[1, 2, 3], keepdim=True)
            loss = (loss / (weight + eps)).sum() / weight.size(0)
        else:
            loss = loss.sum() / (weight.sum() + eps)

    return loss

def masked_loss(loss_func):
    @functools.wraps(loss_func)
    def wrapper(pred,
                target,
                weight=None,
                reduction='mean',
                sample_wise=False,
                **kwargs):
        loss = loss_func(pred, target, **kwargs)
        loss = mask_reduce_loss(loss, weight, reduction, sample_wise)
        return loss

    return wrapper

@masked_loss
def l1_loss(pred, target):
    return F.l1_loss(pred, target, reduction='none')


@masked_loss
def mse_loss(pred, target):
    return F.mse_loss(pred, target, reduction='none')

class L1Loss(nn.Module):
    def __init__(self, loss_weight=100.0, reduction='mean', sample_wise=False):
        super().__init__()

        self.loss_weight = loss_weight
        self.reduction = reduction
        self.sample_wise = sample_wise

    def forward(self, pred, target, weight=None, **kwargs):
        return self.loss_weight * l1_loss(
            pred,
            target,
            weight,
            reduction=self.reduction,
            sample_wise=self.sample_wise)



class MSELoss(nn.Module):
    def __init__(self, loss_weight=100.0, reduction='mean', sample_wise=False):
        super().__init__()
        self.loss_weight = loss_weight
        self.reduction = reduction
        self.sample_wise = sample_wise

    def forward(self, pred, target, weight=None, **kwargs):
        return self.loss_weight * mse_loss(
            pred,
            target,
            weight,
            reduction=self.reduction,
            sample_wise=self.sample_wise)


In [9]:
loss = build_loss(model_config)

In [10]:
import torch.optim as optim

optimizer = optim.AdamW(
    model.parameters(),
    lr=model_config['lr'],
    betas=(0.9, 0.999),
    weight_decay=model_config['weight_decay']
)


In [11]:
import os

import torch.optim as optim
from tqdm import tqdm

from math import cos, pi
from datetime import datetime
class CosineRestartLr(object):
    def __init__(self, base_lr, periods, restart_weights=[1], min_lr=None, min_lr_ratio=None):
        self.periods = periods
        self.min_lr = min_lr
        self.min_lr_ratio = min_lr_ratio
        self.restart_weights = restart_weights
        self.base_lr = base_lr

        self.cumulative_periods = [
            sum(self.periods[0:i + 1]) for i in range(len(self.periods))
        ]

    def annealing_cos(self, start: float, end: float, factor: float, weight: float = 1.) -> float:
        cos_out = cos(pi * factor) + 1
        return end + 0.5 * weight * (start - end) * cos_out

    def get_position_from_periods(self, iteration: int, cumulative_periods):
        for i, period in enumerate(cumulative_periods):
            if iteration < period:
                return i
        raise ValueError(f'Current iteration {iteration} exceeds cumulative_periods {cumulative_periods}')

    def get_lr(self, iter_num, base_lr: float):
        target_lr = self.min_lr if self.min_lr is not None else base_lr

        idx = self.get_position_from_periods(iter_num, self.cumulative_periods)
        current_weight = self.restart_weights[idx]
        nearest_restart = 0 if idx == 0 else self.cumulative_periods[idx - 1]
        current_periods = self.periods[idx]

        alpha = min((iter_num - nearest_restart) / current_periods, 1)
        return self.annealing_cos(base_lr, target_lr, alpha, current_weight)

    def _set_lr(self, optimizer, lr_groups):
        if isinstance(optimizer, dict):
            for k, optim in optimizer.items():
                for param_group, lr in zip(optim.param_groups, lr_groups[k]):
                    param_group['lr'] = lr
        else:
            for param_group, lr in zip(optimizer.param_groups, lr_groups):
                param_group['lr'] = lr

    def get_regular_lr(self, iter_num):
        return [self.get_lr(iter_num, _base_lr) for _base_lr in self.base_lr]

    def set_init_lr(self, optimizer):
        for group in optimizer.param_groups:
            group.setdefault('initial_lr', group['lr'])
            self.base_lr = [group['initial_lr'] for group in optimizer.param_groups]


In [12]:
cosine_lr = CosineRestartLr(model_config['lr'], [model_config['max_iters']], [1], 1e-7)
cosine_lr.set_init_lr(optimizer)
def checkpoint(model, epoch, save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    model_out_path = f"./{save_path}/model_iters_{epoch}.pth"
    torch.save({'state_dict': model.state_dict()}, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

In [13]:
epoch_loss = 0
iter_num = 0
print_freq = 100
save_freq = 10000
Show_freq = 1000

lossList =[]

log_file = "train_log.txt"

while iter_num < model_config['max_iters']:
        now = datetime.now()
        with tqdm(total=print_freq) as bar:
            for feature, label, _ in dataset:        
                if model_config['cpu']:
                    input, target = feature, label
                else:
                    input, target = feature.cuda(), label.cuda()

                regular_lr = cosine_lr.get_regular_lr(iter_num)
                cosine_lr._set_lr(optimizer, regular_lr)

                prediction = model(input)

                optimizer.zero_grad()

                pixel_loss = loss(prediction, target)
                epoch_loss += pixel_loss.item()
                pixel_loss.backward()
                optimizer.step()

                iter_num += 1
                
                bar.update(1)

                if (iter_num % print_freq == 0 or iter_num==100):
                    break

        print("===> Iters({}/{}): Loss: {:.4f}".format(iter_num, model_config['max_iters'], epoch_loss / print_freq))

        with open(log_file, 'a') as logs:
            logs.write("===> Iters({}/{}): Loss: {:.4f}".format(iter_num, model_config['max_iters'], epoch_loss / print_freq) + "\n")

        oneValue = epoch_loss /print_freq
        lossList.append(oneValue)
        if(len(lossList)>10):
            lossList.pop(0)
            sumValue =0
            for kk in range(len(lossList)):
                sumValue += lossList[kk]
            print("===> Average Loss: {:.4f}".format(sumValue / len(lossList)))
            with open(log_file, 'a') as logs:
                logs.write("===> Average Loss: {:.4f}".format(sumValue / len(lossList)) + "\n")

        if ((iter_num % save_freq == 0) or (100==iter_num)):
            checkpoint(model, iter_num, model_config['save_path'])

        if ((iter_num % Show_freq == 0) or (100==iter_num)):
            later = datetime.now()
            difference = (later - now).total_seconds()
            difference_minutes = int(difference // 60)
            difference_seconds = int(difference % 60)
            max_iters = model_config['max_iters']
            expect_difference = difference*(max_iters/iter_num)
            difference_minutes = int(expect_difference // 60)
            difference_seconds = int(expect_difference % 60)

        epoch_loss = 0


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:10<00:00,  9.96it/s]


===> Iters(100/200000): Loss: 0.5959
Checkpoint saved to ././UNet_Inception//model_iters_100.pth


100%|██████████| 100/100 [00:09<00:00, 10.63it/s]


===> Iters(200/200000): Loss: 0.3039


...

===> Iters(199800/200000): Loss: 0.1202
===> Average Loss: 0.1095


100%|██████████| 100/100 [00:05<00:00, 16.79it/s]


===> Iters(199900/200000): Loss: 0.1125
===> Average Loss: 0.1097


100%|██████████| 100/100 [00:04<00:00, 20.26it/s]


===> Iters(200000/200000): Loss: 0.1113
===> Average Loss: 0.1090
Checkpoint saved to ././UNet_Inception//model_iters_200000.pth


In [14]:
# test_config = {
#     'task': 'Congestion',
#     'save_path': './Congestion/',
#     'pretrained': "/home/palaniappan-r/Code/Routing-CNN/UNet_Inception/model_iters_100.pth",
#     'max_iters': 200000,
#     'plot_roc': False,
#     'arg_file': None,
#     'cpu': False,
#     'dataroot': '/home/palaniappan-r/Repos/boosted-UNet/training_datasets/congestion_train_set/congestion',
#     'ann_file_train': './files/train.csv',
#     'ann_file_test': './files/test.csv',
#     'dataset_type': 'CongestionDataset',
#     'batch_size': 4,
#     'aug_pipeline': ['Flip'],
#     'model_type': 'Congestion_Prediction_Net',
#     'in_channels': 3,
#     'out_channels': 1,
#     'lr': 0.0002,
#     'weight_decay': 0.0001,
#     'loss_type': 'MSELoss',
#     'eval_metric': ['NRMS', 'SSIM'],
#     'ann_file': './files/test.csv',
#     'test_mode': True
# }

test_config = {
    'task': 'Congestion',
    'save_path': './Congestion/',
    'pretrained': None,
    'max_iters': 200000,
    'plot_roc': False,
    'arg_file': None,
    'cpu': False,
    'dataroot': '/home/palaniappan-r/Repos/boosted-UNet/training_datasets/congestion_train_set/congestion',
    'ann_file_train': './files/train.csv',
    'ann_file_test': './files/test.csv',
    'dataset_type': 'CongestionDataset',
    'batch_size': 4,
    'aug_pipeline': ['Flip'],
    'model_type': 'Congestion_Prediction_Net',
    'in_channels': 3,
    'out_channels': 1,
    'lr': 0.0002,
    'weight_decay': 0.0001,
    'loss_type': 'MSELoss',
    'eval_metric': ['NRMS', 'SSIM'],
    'ann_file': './files/test.csv',
    'test_mode': True
}


In [15]:
def build_model(model_config):
    model = Congestion_Prediction_Net(test_config)
    model.init_weights()
    model = model.cuda()
    return model


In [16]:
dataset = build_dataset(test_config)
#model = build_model(test_config)
# model = model.cuda()

In [17]:
# Copyright 2022 CircuitNet. All rights reserved.

from __future__ import print_function
import os
import os.path as osp
import json
import numpy as np
import sys
from tqdm import tqdm
from datasets.build_dataset import build_dataset
from utils.metrics import build_metric, build_roc_prc_metric
from utils.configs import Parser


def inference(test_config, model):
    # Build metrics
    metrics = {k:build_metric(k) for k in test_config['eval_metric']}
    avg_metrics = {k:0 for k in test_config['eval_metric']}

    count =0
    with tqdm(total=len(dataset)) as bar:
        for feature, label, label_path in dataset:
            if test_config['cpu']:
                input, target = feature, label
            else:
                input, target = feature.cuda(), label.cuda()

            prediction = model(input)
            for metric, metric_func in metrics.items():
                if not metric_func(target.cpu(), prediction.squeeze(1).cpu()) == 1:
                    avg_metrics[metric] += metric_func(target.cpu(), prediction.squeeze(1).cpu())

            if test_config['plot_roc']:
                save_path = osp.join(test_config['save_path'], 'test_result')
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                file_name = osp.splitext(osp.basename(label_path[0]))[0]
                save_path = osp.join(save_path, f'{file_name}.npy')
                output_final = prediction.float().detach().cpu().numpy()
                np.save(save_path, output_final)
                count +=1

            bar.update(1)


    for metric, avg_metric in avg_metrics.items():
        print("===> Avg. {}: {:.4f}".format(metric, avg_metric / len(dataset))) 
    

In [19]:
inference(test_config, model)

100%|██████████| 3164/3164 [01:06<00:00, 47.89it/s]

===> Avg. NRMS: 0.0473
===> Avg. SSIM: 0.7908
